# Stable Mariages

For this workshop we'll implement a version of the **Nobel Prize-winning algorithm** for the stable marriage problem. It's called the [Gale-Shapley](https://en.wikipedia.org/wiki/Gale%E2%80%93Shapley_algorithm) algorithm AKA the "Deferred Acceptance" algorithm.

The Gale Shapley algorithm solves what's called the **Stable Marriage** Problem. Here is the problem framed as marrying men and women: 

> Given $n$ men and $n$ women, where each person has ranked all members of the opposite sex in order of preference, marry the men and women together such that there are no two people of opposite sex who would both rather have each other than their current partners. When there are no such pairs of people, the set of marriages is deemed stable.

This little model however is generally applied to more than marrying men and women. For example, it's the algorithm used to match newly graduated doctors with hospitals, and a variation on it matches kidney patients with organ donors in our hospital system.

The algorithmic question is, given lists of preferences as input, can we find a stable marriage? Can we even guarantee a stable marriage will exist for any set of preferences? The answer to both questions is yes, and it uses an algorithm called deferred acceptance.

Here is an informal description of the algorithm. It goes in rounds. In each round, each man “proposes” to the highest-preferred woman that has not yet rejected him. On the other side, each woman holds a reference to a man at all times. If a woman gets new proposals in a round, she immediately rejects every proposer except her most preferred, but does not accept that proposal. She “defers” the acceptance of the proposal until the very end.

![](Gale-Shapley.gif)

# Your Task

Build a solution `gale_shapley(men, women) -> dict` Where the input is a list of suitors and a list of Suiteds with every one in these lists holding their lists of preferences. The output is a dictionary `suitor -> suited`

Here is a possible way to design a solution:

### Man Class

The `Man(id, preference_list)` class holds the following attributes:

- Its own ID (an `int`)

- A list or array of IDs, which are ordered. So `preference_list[0]` is prefered to `preference_list[1]` and so on

- A method `Suitor.preference()` which points to its current possible reference. It should start by pointing at `preference_list[0]` and every time the Suitor gets rejected in the algorithm, this should point to the next preference.

### Woman Class

The `Woman(id, preference_list)` class holds the following attributes:

- Its own ID (an `int`)

- A list or array of IDs, which are ordered. So `preference_list[0]` is prefered to `preference_list[1]` and so on

- A set of current suitors

- A method `Suited.reject()` which returns all current suitors except the most preferred one

### The Stable Mariage Algorithm

Takes in a list of men and women, loops over suitors trying to find a match until there aren't any unassigned suitors left. Here is the pseudocode for the algorithm:

```
algorithm stable_matching
    Initialize all m ∈ M and w ∈ W to free
    while ∃ free man m who still has a woman w to propose to do
        w := first woman on m's list to whom m has not yet proposed
        if w is free then
            (m, w) become engaged
        else some pair (m', w) already exists
            if w prefers m to m' then
                m' becomes free
                (m, w) become engaged 
            else
                (m', w) remain engaged
            end if
        end if
    repeat
```


Here is some example code of a solution working:

```
men = [
    Man(id=0, preference_list=[0,1,2,3]),
    Man(id=1, preference_list=[2,3,0,1]),
    Man(id=2, preference_list=[1,0,3,2]),
    Man(id=3, preference_list=[3,2,1,0]),
]

women = [
    Woman(id=0, preference_list=[0,1,2,3]),
    Woman(id=1, preference_list=[2,3,0,1]),
    Woman(id=2, preference_list=[1,0,3,2]),
    Woman(id=3, preference_list=[3,2,1,0]),
]

stable_marriage(men, women)
```

output:

```
{0: 0, 
 2: 1, 
 1: 2, 
 3: 3}
```

In [1]:
# reference:
# https://jeremykun.com/2014/04/02/stable-marriages-and-designing-markets/
# a-programmers-introduction-to-mathematics, pg. 56-57

In [36]:
# Jeremy Kun Solution

class Man(object):
    def __init__(self, id, preference_list):
        self.prefList = preference_list
        self.rejections = 0 # num rejections is also the index of the next option
        self.id = id
    
    def preference(self):
        return self.prefList[self.rejections]
    
    def __repr__(self):
        return repr(self.id)


class Woman(object):
    def __init__(self, id, preference_list):
        self.prefList = preference_list
        self.held = None
        self.currentSuitors = set()
        self.id = id

    def reject(self):
        if len(self.currentSuitors) == 0:
            return set()

        if self.held is not None:
            self.currentSuitors.add(self.held)

        self.held = min(self.currentSuitors, key=lambda suitor: self.prefList.index(suitor.id))
        rejected = self.currentSuitors - set([self.held])
        self.currentSuitors = set()
        
        return rejected
    
    def __repr__(self):
        return repr(self.id)


def StableMarriage(men, women):
    unassigned = set(men)
    
    while len(unassigned) > 0:
        for suitor in unassigned:
            women[suitor.preference()].currentSuitors.add(suitor)
            unassigned = set()

        for suited in women:
            unassigned |= suited.reject()

        for suitor in unassigned:
            suitor.rejections += 1

    return dict([(suited.held, suited) for suited in women])

In [37]:
men = [
    Man(id=0, preference_list=[0,4,1,2,3]),
    Man(id=1, preference_list=[4,2,3,0,1]),
    Man(id=2, preference_list=[4,1,0,3,2]),
    Man(id=3, preference_list=[4,3,2,1,0]),
    Man(id=4, preference_list=[4,2,3,1,0]),
]

women = [
    Woman(id=0, preference_list=[0,1,4,2,3]),
    Woman(id=1, preference_list=[4,2,3,0,1]),
    Woman(id=2, preference_list=[1,0,3,4,2]),
    Woman(id=3, preference_list=[4,3,2,1,0]),
    Woman(id=4, preference_list=[3,4,2,1,0]),
]

StableMarriage(men, women)

{0: 0, 2: 1, 1: 2, 4: 3, 3: 4}

In [38]:
class Man():
    def __init__(self, m_id, preference_list):
        self.id = int(m_id)
        self.preference_list = preference_list
        
    def preference(self):
        return self.preference_list[0]
        
    def __repr__(self):
        return repr(self.id)
    
class Woman():
    def __init__(self, w_id, preference_list):
        self.id = int(w_id)
        self.preference_list = preference_list
        self.currentsuitor = set()
        
    def __repr__(self):
        return repr(self.id)

In [ ]:
def stable_marriage(men, women):
    free_men = []
    currentlymatched = []
    
    for m in men:
        free_men.append(m)

    while len(free_men) > 0:
        for m in free_men:
            for w in m.preference_list():
                taken = [couple for couple in currentlymatched if w in couple]
                if (len(taken)==0):
                    currentlymatched.append([m,w])
                    free_men.remove(m)
                    set(free_men)
                    print(m)
                    break
                elif (len(taken)>0):
                    suited = women[w].preference_list.index(taken[0][0])
                    suitor = women[w].preference_list.index(m)
                    if (suited > suitor):
                        free_men.remove(m)
                        free_men.append(taken[0][0])
        return currentlymatched

stable_marriage(men, women)